In [14]:
# from https://github.com/evansherlock/nytimesarticle/blob/master/nytimesarticle.py

import requests

API_ROOT = 'http://api.nytimes.com/svc/search/v2/articlesearch.'

API_SIGNUP_PAGE = 'http://developer.nytimes.com/docs/reference/keys'

class NoAPIKeyException(Exception):
    def __init__(self, value):
        self.value = value
    def __str__(self):
        return repr(self.value)

class articleAPI(object):
    def __init__(self, key = None):
        """
        Initializes the articleAPI class with a developer key. Raises an exception if a key is not given.
        
        Request a key at http://developer.nytimes.com/docs/reference/keys
        
        :param key: New York Times Developer Key
        
        """
        self.key = key
        self.response_format = 'json'
        
        if self.key is None:
            raise NoAPIKeyException('Warning: Missing API Key. Please visit ' + API_SIGNUP_PAGE + ' to register for a key.')
    
    def _utf8_encode(self, d):
        """
        Ensures all values are encoded in UTF-8 and converts them to lowercase
        
        """
        for k, v in d.items():
            if isinstance(v, str):
                d[k] = v.encode('utf8').lower()
            if isinstance(v, list):
                for index,item in enumerate(v):
                    item = item.encode('utf8').lower()
                    v[index] = item
            if isinstance(v, dict):
                d[k] = self._utf8_encode(v)
        
        return d
    
    def _bool_encode(self, d):
        """
        Converts bool values to lowercase strings
        
        """
        for k, v in d.items():
            if isinstance(v, bool):
                d[k] = str(v).lower()
        
        return d

    def _options(self, **kwargs):
        """
        Formats search parameters/values for use with API
        
        :param \*\*kwargs: search parameters/values
        
        """
        def _format_fq(d):
            for k,v in d.items():
                if isinstance(v, list):
                    d[k] = ' '.join(map(lambda x: '"' + x.decode('utf-8') + '"', v))
                else:
                    d[k] = '"' + v.decode('utf-8') + '"'
            values = []
            for k,v in d.items():
                value = '%s:(%s)' % (k,v)
                values.append(value)
            values = ' AND '.join(values)
            return values
        
        kwargs = self._utf8_encode(kwargs)
        kwargs = self._bool_encode(kwargs)
        
        values = ''
        
        for k, v in kwargs.items():
            if k is 'fq' and isinstance(v, dict):
                v = _format_fq(v)
            elif isinstance(v, list):
                v = [x.decode('utf-8') for x in v]
                v = ','.join(v)
            values += '%s=%s&' % (k, v)
        
        return values

    def search(self, 
                response_format = None, 
                key = None, 
                **kwargs):
        """
        Calls the API and returns a dictionary of the search results
        
        :param response_format: the format that the API uses for its response, 
                                includes JSON (.json) and JSONP (.jsonp). 
                                Defaults to '.json'.
                                
        :param key: a developer key. Defaults to key given when the articleAPI class was initialized.
        
        """
        if response_format is None:
            response_format = self.response_format
        if key is None:
            key = self.key
        
        url = '%s%s?%sapi-key=%s' % (
            API_ROOT, response_format, self._options(**kwargs), key
        )
        
        r = requests.get(url)
        return r.json()

In [28]:
key = "637ca5fe3c8748a6b33d0d3079fb4d30"
api = articleAPI(key)
articles = api.search( q = 'Microsoft', begin_date = 20180701)
# , facet_field = ['source','day_of_week'], facet_filter = True 
# fq = {'source':['Reuters','AP', 'The New York Times']}

In [59]:
articles['response']['docs'][1]['web_url']

'https://www.nytimes.com/reuters/2018/09/18/technology/18reuters-microsoft-dynamics365.html'

In [60]:
data = [el['web_url'] for el in articles['response']['docs']]

In [61]:
data

['/interactive/2018/07/19/business/dealbook/db-eufine-msft1.html',
 'https://www.nytimes.com/reuters/2018/09/18/technology/18reuters-microsoft-dynamics365.html',
 'https://www.nytimes.com/reuters/2018/09/17/business/17reuters-github-m-a-microsoft-eu.html',
 'https://www.nytimes.com/2018/07/13/technology/microsoft-facial-recognition.html',
 'https://www.nytimes.com/2018/07/19/technology/microsoft-earnings-cloud-computing.html',
 'https://www.nytimes.com/2018/07/26/technology/microsoft-ice-immigration.html',
 'https://www.nytimes.com/aponline/2018/09/19/business/ap-financial-markets-stocks.html',
 'https://www.nytimes.com/2018/08/21/us/politics/russia-cyber-hack.html',
 'https://www.nytimes.com/2018/08/21/briefing/microsoft-primary-election-epa.html',
 'https://www.nytimes.com/podcast/2007/01/30/washington/30backstory-peters.html']

In [ ]:
API_ROOT = 'http://api.nytimes.com/svc/search/v2/articlesearch.'
API_SIGNUP = 'http://developer.nytimes.com/docs/reference/keys'

In [ ]:
#req = requests.get(API_ROOT)

In [ ]:
from requests.auth import HTTPDigestAuth
data = q = 'Obama', fq = {'headline':'Obama', 'source':['Reuters','AP', 'The New York Times']}, begin_date = 20111231, facet_field = ['source','day_of_week'], facet_filter = True
url = '%s%s?%sapi-key=%s' % (API_ROOT,'json', data,key)
r = requests.get(url,key)

In [ ]:
r = requests.get(API_SIGNUP,key)

In [ ]:
d = requests.session()